In [0]:
##
access_key = 'xxxxxxxxxx'
secret_key = 'xxx+xxxxxxxx'
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", access_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", secret_key)

# If you are using Auto Loader file notification mode to load files, provide the AWS Region ID.
aws_region = "us-east-2"
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3." + aws_region + ".amazonaws.com")



In [0]:
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, to_date


In [0]:
def read_file(path):
    df = spark.read.csv(path,inferSchema=True,header=True)
    return df
#Example#
#df = spark.read.csv('s3a://techykuntal-demo-data-bucket/employee/*',inferSchema=True,header=True)



In [0]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- client_id: integer (nullable = true)
 |-- card_brand: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- card_number: long (nullable = true)
 |-- expires: string (nullable = true)
 |-- cvv: integer (nullable = true)
 |-- has_chip: string (nullable = true)
 |-- num_cards_issued: integer (nullable = true)
 |-- credit_limit: string (nullable = true)
 |-- acct_open_date: string (nullable = true)
 |-- year_pin_last_changed: integer (nullable = true)
 |-- card_on_dark_web: string (nullable = true)



In [0]:
df.count()

Out[93]: 6146

In [0]:
def measure_time(func, *args):
    start_time = time.time()
    result = func(*args)
    end_time = time.time()
    print(f"Tempo de execução para {func.__name__}: {end_time - start_time:.4f} segundos")
    return result

In [0]:
def clean_dataframe(df):
    df = df.withColumn("has_chip", regexp_replace(col("has_chip"), "YES", "True"))
    df = df.withColumn("has_chip", regexp_replace(col("has_chip"), "NO", "False"))
    df = df.withColumn("credit_limit", regexp_replace(col("credit_limit"), "\\$", "").cast("integer"))
    df = df.withColumn("expires", to_date(col("expires"), "MM/yyyy"))
    df = df.withColumn("acct_open_date", to_date(col("acct_open_date"), "MM/yyyy"))
    return df

In [0]:

# Função para ordenar o DataFrame
def sort_dataframe(df):
    return df.orderBy(col("credit_limit").desc())

In [0]:
# Função para agrupar o DataFrame
# Exemplo: Contar quantos cartões de cada marca existem
def group_dataframe(df):
    return df.groupBy("card_brand").count()

In [0]:
# Função para filtrar o DataFrame
# Exemplo: Filtrar cartões que estão na dark web e têm chip
def filter_dataframe(df):
    return df.filter((col("card_on_dark_web") == "No") & (col("has_chip") == "True"))


In [0]:
def save_file(df):
    return df.write \
    .option("header", True) \
    .csv('output_path.csv')

In [0]:
# Aplicar as transformações com medição de tempo
df = measure_time(read_file, 's3a://financial-dataset-ada/cards_data.csv')
df_cleaned = measure_time(clean_dataframe, df)
df_sorted = measure_time(sort_dataframe, df_cleaned)
df_grouped = measure_time(group_dataframe, df_cleaned)
df_filtered = measure_time(filter_dataframe, df_cleaned)
df_save = measure_time(save_file, df)

Tempo de execução para read_file: 1.5573 segundos
Tempo de execução para clean_dataframe: 0.1067 segundos
Tempo de execução para sort_dataframe: 0.0194 segundos
Tempo de execução para group_dataframe: 0.0174 segundos
Tempo de execução para filter_dataframe: 0.0178 segundos
Tempo de execução para save_file: 1.9432 segundos


In [0]:
# Mostrar os resultados
df.show()
df_cleaned.show()
df_sorted.show()
df_grouped.show()
df_filtered.show()

+----+---------+----------+---------------+----------------+-------+---+--------+----------------+------------+--------------+---------------------+----------------+
|  id|client_id|card_brand|      card_type|     card_number|expires|cvv|has_chip|num_cards_issued|credit_limit|acct_open_date|year_pin_last_changed|card_on_dark_web|
+----+---------+----------+---------------+----------------+-------+---+--------+----------------+------------+--------------+---------------------+----------------+
|4524|      825|      Visa|          Debit|4344676511950444|12/2022|623|     YES|               2|      $24295|       09/2002|                 2008|              No|
|2731|      825|      Visa|          Debit|4956965974959986|12/2020|393|     YES|               2|      $21968|       04/2014|                 2014|              No|
|3701|      825|      Visa|          Debit|4582313478255491|02/2024|719|     YES|               2|      $46414|       07/2003|                 2004|              No|
|  4